In [2]:
# -- coding: utf-8 --
from Environment import Environment
from model import First_procedure
from model import Second_procedure
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm
import pandas as pd
import seaborn as sbn
import numpy as np
from matplotlib.colors import ListedColormap
import copy
import pickle
import time
from multiprocessing import Pool
import random


In [3]:
'''
There are two class of parameters:

First parameters class are the parameters of the first-process--generate the anchor point, denoted by the inital letter 'f' 
f_time_distribution : type=str, the time displacement distribution of the first procedure. The parameters can choose ['u','exp','power']
f_time_parameters : type=[], the list of the parameters of the corresponded time distribution. for 'exp','power', f_time_parameters=[num],num<0
f_space_distribution : anchor point preferential choice of the center. The parameters can choose ['u','exp','power']
f_space_parameters = : the parameters of the f_space_distribution

Second are the parameters of the second-process -- expand the anchor point area and population,denoted by the inital letter 'f'
s_time_distribution: type=str, the time displacement distribution of the first procedure. The parameters can choose ['u','exp','power']
s_time_parameters: type=[], the list of the parameters of the corresponded time distribution. for 'exp','power', f_time_parameters=[num],num<0
step_r: the step distance every step, step_r=0.5 default
'''

def main(main_path,r,alpha,stop_step=1000000,memory=100000):

    f_time_distribution = 'exp'
    f_time_parameters = [1]
    f_space_distribution = 'u'
    f_space_parameters = [1]


    s_time_distribution = 'exp'
    s_time_parameters = [alpha]
    step_r=.5

    r=r
    ####--init the model-- ####
    first_process=First_procedure(f_time_parameters=f_time_parameters,f_time_distribution=f_time_distribution,f_space_parameters=f_space_parameters,f_space_distribution= f_space_distribution)
    second_process=Second_procedure(s_time_distribution,s_time_parameters,step_r)
    Envir = Environment(shape='square', r=r,Memory=memory)

    anchor_point_flag=1
    t_now=0
    next_second_T=0
    next_first_T=0

    first_anchorpoint=first_process.cal(emptry_grid_list=Envir.get_empty_gridlist())
    flag=anchor_point_flag
    Envir.add_point(first_anchorpoint,flag=flag)
    Envir.anchorPoint.append(first_anchorpoint)

    simulation_clock=0


    #record invasion:
    change_steps=[]
    last_max_id=-1
    max_step=stop_step
    t_all=[0]
    flag_dict={}
    while(True):
        simulation_clock+=1
        next_second_T=second_process.get_next_T(Envir.point_num)
        next_first_T=first_process.get_next_T(Envir.get_anchorpoint_number())
        if(next_second_T>=next_first_T):
            t_now=next_first_T
        else:
            t_now=next_second_T

        if  t_now==next_first_T:
            ## first procedure
            anchor_point_flag += 1
            anchorpoint=first_process.cal(emptry_grid_list=Envir.get_empty_gridlist())
            if(anchorpoint==None):
                break
            flag=anchor_point_flag
            Envir.anchorPoint.append(anchorpoint)
            Envir.add_point(anchorpoint,flag)
        if t_now==next_second_T:
            point,flag = second_process.cal(gridDict=Envir.gridDict,rand_point=Envir.get_random_point())
            if point:
                Envir.add_point(point,flag=flag)
                # is invasion
                if(flag not in flag_dict):
                    flag_dict[flag]=0
                else:
                    flag_dict[flag]+=1
                pop_distribuition=flag_dict
                sorted_pop_distribution=sorted(pop_distribuition.items(),key=lambda x:x[0],reverse=True)
              
                if(sorted_pop_distribution[0][0]!=last_max_id):
                    t_sum=t_all[-1]+t_now
                    t_all.append(t_sum)
                    change_steps.append(simulation_clock)
                    last_max_id=sorted_pop_distribution[0][0]
                
                
        if(simulation_clock%(int(stop_step/50))==0):
            print (simulation_clock)
        if(simulation_clock>max_step):
            break

    file_name=main_path+'/'+str(time.mktime(time.localtime())+random.randint(1,100000))+'.pkl'
    values=[t_all,change_steps]
    with open(file_name,'wb+') as file:
        pickle.dump(values,file=file)
        print ('OK')


In [ ]:
if __name__=='__main__':
    main_path='/home/wangjianying/data/Spatial_Yule/change_rate'
    alphalist=[2,5,10,20,50]
    for alpha in alphalist:
        r=100
        print (alpha)
        simulate_num=100
        pool_num=40
        Pool_tool=Pool(pool_num)
        stop_step=10000000
        memory=stop_step/10
        for i in range(simulate_num):
            value=Pool_tool.apply_async(main,args=(main_path,r,alpha,stop_step,memory))
            print('Waiting for all subprocesses done...')
        Pool_tool.close()
        Pool_tool.join() 
        print('All subprocesses done.')
        
        import os 
        parts=os.listdir(main_path)
        paths=[main_path+'//'+path for path in parts if 'pkl' in path and str(stop_step)+'.pkl' not in path]
        temp_list=[]
        for path in paths:
            with open(path,'rb+') as file:
                print (path)
                temp_tempvalue=pickle.load(file)
                os.remove(path)
                temp_list.append(temp_tempvalue)
        outpath=main_path+'//'+str(alpha)+'_'+str(stop_step)+r'.pkl'
        with open(outpath,'wb+') as file:
            pickle.dump(temp_list,file)
        print ('All')

2
Waiting for all subprocesses done...
Waiting for all subprocesses done...
Waiting for all subprocesses done...
Waiting for all subprocesses done...
Waiting for all subprocesses done...
Waiting for all subprocesses done...
Waiting for all subprocesses done...
Waiting for all subprocesses done...
Waiting for all subprocesses done...
Waiting for all subprocesses done...
Waiting for all subprocesses done...
Waiting for all subprocesses done...
Waiting for all subprocesses done...
Waiting for all subprocesses done...
Waiting for all subprocesses done...
Waiting for all subprocesses done...
Waiting for all subprocesses done...
Waiting for all subprocesses done...
Waiting for all subprocesses done...
Waiting for all subprocesses done...
Waiting for all subprocesses done...
Waiting for all subprocesses done...
Waiting for all subprocesses done...
Waiting for all subprocesses done...
Waiting for all subprocesses done...
Waiting for all subprocesses done...
Waiting for all subprocesses done...

3000000
3000000
3000000
3000000
3000000
3000000
3000000
3000000
3000000
3000000
3000000
3000000
3000000
3000000
3000000
3200000
3200000
3200000
3200000
3000000
3200000
3200000
3200000
3000000
3200000
3200000
3200000
3200000
3200000
3000000
3200000
3200000
3200000
3200000
3200000
3200000
3200000
3200000
3200000
3200000
3200000
3200000
3200000
3200000
3200000
3200000
3200000
3200000
3200000
3200000
3200000
3200000
3400000
3200000
3400000
3400000
3200000
3200000
3200000
3400000
3400000
3400000
3400000
3400000
3200000
3400000
3400000
3400000
3400000
3400000
3400000
3200000
3400000
3400000
3400000
3400000
3400000
3400000
3400000
3400000
3400000
3400000
3400000
3400000
3400000
3400000
3400000
3400000
3400000
3400000
3400000
3400000
3600000
3400000
3600000
3600000
3400000
3600000
3400000
3400000
3600000
3600000
3600000
3400000
3600000
3600000
3600000
3600000
3600000
3600000
3600000
3400000
3600000
3600000
3600000
3600000
3600000
3600000
3600000
3600000
3600000
3600000
3600000
3600000
3600000


8400000
8200000
8200000
8000000
8200000
8000000
8200000
8000000
8000000
8000000
8000000
8200000
7800000
8200000
8000000
8400000
7800000
8400000
8400000
8600000
8400000
8200000
8400000
8200000
8000000
8200000
8600000
8600000
8200000
8400000
8400000
8200000
8200000
8400000
8600000
8400000
8200000
8200000
8400000
8200000
8400000
8400000
8200000
8200000
8400000
8200000
8400000
8200000
8400000
8400000
8200000
8400000
8400000
8200000
8600000
8600000
8000000
8000000
8600000
8400000
8600000
8400000
8800000
8800000
8600000
8400000
8800000
8200000
8400000
8600000
8600000
8400000
8600000
8400000
8600000
8600000
8800000
8400000
8400000
8600000
8400000
8400000
8600000
8400000
8800000
8600000
8600000
8400000
8600000
8400000
8600000
8400000
8800000
8400000
8600000
8600000
8200000
9000000
8600000
8800000
8400000
8600000
8200000
9000000
8800000
8800000
8600000
9000000
9000000
8600000
8800000
8600000
8800000
8800000
8600000
8800000
8600000
8600000
8600000
8800000
8800000
8600000
8800000
8600000
8800000


2200000
3200000
2800000
600000
3600000
2400000
3000000
5000000
4000000
1400000
4800000
3600000
4200000
3800000
3600000
3600000
2800000
3000000
4000000
4400000
5200000
2800000
1800000
3600000
1000000
2200000
4000000
3000000
5600000
3400000
3200000
2800000
3800000
800000
2800000
4200000
4400000
3200000
2600000
3200000
5000000
3400000
2400000
5200000
3800000
4400000
2600000
3000000
4200000
5200000
3200000
3400000
1600000
3800000
4000000
3800000
4600000
3800000
3000000
1000000
3000000
1200000
3600000
5400000
3400000
2400000
4200000
4000000
2000000
3200000
5800000
4200000
3000000
3800000
2800000
2800000
3400000
5200000
3000000
4400000
4600000
2600000
4000000
4600000
3400000
5400000
3600000
3200000
1800000
5400000
4000000
3600000
4400000
3400000
4000000
1200000
4200000
1400000
4800000
2600000
5600000
3400000
3200000
3800000
3600000
4000000
2200000
4400000
6000000
3000000
4400000
3200000
4200000
3600000
3200000
3000000
2000000
4800000
5600000
3200000
4200000
4800000
5400000
3600000
4600000
40